# Data Preprocessing
We do it in python, because there is a much better support for stuff like multilingual bert models from huggingface.

In [1]:
import csv
import re
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
import json
import gc


/home/benjamin/miniconda3/envs/pytorch/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load the dataset

In [2]:
input_filename = "settles.acl16.learning_traces.13m.csv"  # Change to the name of your input CSV file


In [3]:
vocabulary_lex = ""

with open(input_filename, 'r') as csv_file:
    reader = csv.reader(csv_file)
    header = next(reader)  # Read the header
    vocabulary_lex = set([column[7] for column in reader])
    del header, reader
del csv_file
gc.collect()


20

In [4]:
all_lex_tags = set()

for lex in vocabulary_lex:
    for l in lex.split("/")[-1].split("<")[1:]:
        all_lex_tags.add(l.replace(">", ""))
    
print(all_lex_tags)


{'cni', 'ref', 'vbmod', '@cnj:alors_que', 'vbser', 'n+wehr', '@prn:celui_que', 'pl+gli', 'sw', 'mf', '@prn:le_mien', '@adv:tout_a_fait', '*gndr', 'pr+ele', 'p3', 'vbdo', 'ifi', 'f', 'dim', '@cnj:bien_que', 'p1', '@cnj:depuis_que', 'n+ende', '@pr:a_cote_de', '@adv:a_posteriori', 'sg+mi', 'adj', 'pr', 'rel', 'ant', 'pp', 'obj', 'prs', 'past', '@pr:au_dela_de', 'n+hof', '@ij:au_revoir', 'enc', '@pr:pres_de', 'pred', '@prn:le_tien', '@det:de_le', 'def', 'tn', '@common_phrases:a_demain', '@cnj:tandis_que', 'pr+o', 'nn', '@neg:il_ne_y_a', 'preadv', 'imp', 'predet', '@n:petit_ami', 'vblex', '@cnj:avant_que', '@prn:ce_dont', 'comp', 'm', '@past', '@pr:un_peu_de', '@adv:en_general', '@prn:le_notre', 'sg', '@prn:l_un', '@cnj:des_que', '@neg:pas_du_tout', 'inf+ci', '@pr:plus_de', '@ij:buenas_noches', 'pr+der', 'inf', 'ord', '@cond', 'pii', '@cnj:autant_que', 'adv', '@adv:por_supuesto', 'det', 'sint', '*numb', 'pis', '@common_phrases:de_rien', 'fti', '@adv:au_moins', '@prn:le_meme', 'qnt', 'pr+das

## Let's do the Bert Embeddings

In [5]:
pattern = r'([^<]+/\w+)<'

vocabulary = set()
for i in [re.match(pattern, s).group(1) for s in vocabulary_lex if re.match(pattern, s)]:
    words = i.split("/")
    vocabulary.add(words[0])
    vocabulary.add(words[1])
vocabulary= list(vocabulary)
len(vocabulary)

11198

In [6]:
# Load pre-trained multilingual BERT model
tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')
model = AutoModel.from_pretrained('bert-base-multilingual-cased')


def get_embeddings_iteratively(phrases, batch_size=2, output_file='embeddings.npy'):
    all_embeddings = []
    for i in range(0, len(phrases), batch_size):
        batch = phrases[i:i + batch_size]
        inputs = tokenizer(batch, return_tensors='pt', padding=True, truncation=True)
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1)  # Mean pooling
        embeddings = embeddings.detach().numpy()
        all_embeddings.append(embeddings)
        del inputs, outputs, embeddings
        torch.cuda.empty_cache()  # Clear GPU memory if using GPU
    all_embeddings = np.vstack(all_embeddings)
    np.save(output_file, all_embeddings)
    return all_embeddings


In [7]:
embeddings = get_embeddings_iteratively(vocabulary, batch_size=1000, output_file='embeddings.npy')


In [8]:
embeddings = np.load('embeddings.npy')
embeddings.shape

(11198, 768)

In [9]:
# Reduce dimensionality of embeddings
pca = PCA(n_components=2)
reduced_embeddings = pca.fit_transform(embeddings)

# plt.figure(figsize=(15, 15))
# for i, phrase in enumerate(vocabulary):
#     plt.scatter(reduced_embeddings[i, 0], reduced_embeddings[i, 1], label=phrase, s=3)
#     plt.text(reduced_embeddings[i, 0], reduced_embeddings[i, 1], phrase, fontsize=7, alpha=0.4, ha='right', va='center')
# plt.title("BERT Embeddings in 2D")
# plt.xlabel("PCA Component 1")
# plt.ylabel("PCA Component 2")
# plt.show()



## Export the calculated PCA embeddings

In [10]:
vocabulary_dataset = []
for i, word in enumerate(vocabulary):
    vocabulary_dataset.append({
        "word": word,
        "position": list(reduced_embeddings[i])
    })

def convert_to_serializable(obj):
    if isinstance(obj, np.float32):
        return float(obj)  # Convert to Python float
    else:
        raise TypeError("Type not serializable")


with open("../src/data/vocabulary_dataset.json", "w") as f:
    json.dump(vocabulary_dataset, f, default=convert_to_serializable, indent=4)


In [11]:
vocabulary_dataset

[{'word': 'bilingual', 'position': [-1.4521351, 0.6127987]},
 {'word': 'sais', 'position': [1.0359614, -0.75902843]},
 {'word': 'zitat', 'position': [0.8882726, -3.0880938]},
 {'word': 'pasos', 'position': [-2.3521097, 0.8898789]},
 {'word': 'donnent', 'position': [3.6974194, 3.567543]},
 {'word': 'ailes', 'position': [-0.08706269, 0.91799426]},
 {'word': 'würde', 'position': [5.3186345, -1.3270471]},
 {'word': 'besten', 'position': [1.1196035, -3.1323419]},
 {'word': 'estuviste', 'position': [-0.45856893, -0.33407643]},
 {'word': 'determine', 'position': [5.578612, 3.4223497]},
 {'word': 'centre', 'position': [-1.3668084, 0.31434205]},
 {'word': 'âme', 'position': [-0.71928316, 1.8840284]},
 {'word': 'reduce', 'position': [1.2919526, 0.64490634]},
 {'word': 'autoren', 'position': [0.34962726, -1.5800176]},
 {'word': 'käufer', 'position': [1.5957152, -3.1689541]},
 {'word': 'empresa', 'position': [-1.4223624, 1.4041125]},
 {'word': 'expresión', 'position': [1.6312529, 2.364848]},
 {'wo

In [ ]:
# with open(output_file, 'w', newline='') as csv_file:
#     writer = csv.writer(csv_file)
#     writer.writerow(header)  # Write the header
#     writer.writerows(data)   # Write the filtered data
